In [2]:
import pandas as pd
import numpy as np
from os import path, environ

Берём большой Датасет

In [3]:
data_dir = "C://Users//asus//Desktop//Netology//Data//Last_FM"
environ["DATA_DIR"] = "/home/spynal/DATA/lastfm-dataset-360K/"
col_names = ["user", "artist-mbid", "artist-name", "total-plays"]
data = pd.read_csv(
    path.join(data_dir, "lastfm_small.tsv"),
    sep="\t",
    header=None,
    names=col_names
)

Пустые значения заменяем 0

In [4]:
data.fillna(0, inplace=True)

In [5]:
data['user_id']=data['user'].astype('category').cat.codes.copy()+1
data['artist_id']=data['artist-mbid'].astype('category').cat.codes.copy()+1

In [6]:
data.drop(['user','artist-mbid','artist-name'], axis=1,inplace=True)

Делим на Train\Test

In [7]:
from sklearn.model_selection import train_test_split

In [8]:
data_Test, data_Train=train_test_split(data, test_size=0.2, random_state=0 )

Записываем в файлы, для MREC

In [9]:
test_file_name = "lastfm.test.0"
data_Test[["user_id", "artist_id", "total-plays"]].to_csv(
    path.join(data_dir,test_file_name),
    sep="\t",
    header=False,
    index=False
)
train_file_name = "lastfm.train.0"
data_Train[["user_id", "artist_id", "total-plays"]].to_csv(
    path.join(data_dir,train_file_name),
    sep="\t",
    header=False,
    index=False
)

Конвертим в разряженную матрицу

In [10]:
from scipy.sparse import csr_matrix

def sparse_info(sparse_matrix: csr_matrix) -> None:
    print("Размерности матрицы: {}".format(sparse_matrix.shape))
    print("Ненулевых элементов в матрице: {}".format(sparse_matrix.nnz))
    print("Доля ненулевых элементов: {}"
          .format(sparse_matrix.nnz / sparse_matrix.shape[0] / sparse_matrix.shape[1])
    )
    print("Среднее значение ненулевых элементов: {}".format(sparse_matrix.data.mean()))
    print("Максимальное значение ненулевых элементов: {}".format(sparse_matrix.data.max()))
    print("Минимальное значение ненулевых элементов: {}".format(sparse_matrix.data.min()))

In [11]:
from scipy.sparse import coo_matrix
plays_knn=coo_matrix((data_Train['total-plays'].astype(np.float32),(data_Train['artist_id'], data_Train['user_id'])))
sparse_info(plays_knn.tocsr())

Размерности матрицы: (66799, 20466)
Ненулевых элементов в матрице: 199998
Доля ненулевых элементов: 0.00014629272240249542
Среднее значение ненулевых элементов: 219.34535217285156
Максимальное значение ненулевых элементов: 134993.0
Минимальное значение ненулевых элементов: 1.0


In [12]:
from scipy.sparse import coo_matrix
plays_als=coo_matrix((data_Train['total-plays'].astype(np.double),(data_Train['artist_id'], data_Train['user_id'])))
sparse_info(plays_als.tocsr())

Размерности матрицы: (66799, 20466)
Ненулевых элементов в матрице: 199998
Доля ненулевых элементов: 0.00014629272240249542
Среднее значение ненулевых элементов: 219.34536845368453
Максимальное значение ненулевых элементов: 134993.0
Минимальное значение ненулевых элементов: 1.0


Берём ALS и KNN

In [13]:
from implicit.nearest_neighbours import CosineRecommender
from implicit.als import AlternatingLeastSquares

In [14]:
model_knn=CosineRecommender()
model_knn.fit(plays_knn)
sparse_info(model_knn.similarity)

Размерности матрицы: (66799, 66799)
Ненулевых элементов в матрице: 481101
Доля ненулевых элементов: 0.00010781925771473777
Среднее значение ненулевых элементов: 0.3207606778610867
Максимальное значение ненулевых элементов: 1.0000000585911983
Минимальное значение ненулевых элементов: 0.0


In [15]:
%%time
model_als_default=AlternatingLeastSquares()
model_als_default.fit(plays_als)

Wall time: 13 s


После нескольких попыток на меньшем датасете, вырбрал новые параметры для ALS

In [16]:
%%time
model_als_tune=AlternatingLeastSquares(factors=20,regularization=0.01, iterations=300)
model_als_tune.fit(plays_als)

Wall time: 42.2 s


In [22]:
user_plays = plays_knn.T.tocsr()

In [25]:
model_knn.recommend('1533', user_plays)

[(20058, 296.81502269772722),
 (14830, 191.90314544230824),
 (64624, 170.37926258945791),
 (48303, 145.75837560737025),
 (4141, 135.68269031543178),
 (63824, 112.24380670497186),
 (7015, 112.24380670497186),
 (20027, 109.05364214562719),
 (15452, 108.4191248822043),
 (42028, 104.2125582274963)]

In [23]:
data_Test["user_id"].unique()

array([15568,  1615, 15787, ...,    59,  9814, 18894], dtype=int64)

In [37]:
%%time
user_plays = plays_knn.T.tocsr()
with open((data_dir+"/knn/lastfm.test.0.recs.tsv"), "w") as output_file:
    for user_id in data_Test["user_id"].unique():
        for artist_id, score in model_knn.recommend(user_id, user_plays):
                output_file.write("%s\t%s\t%s\n" % (user_id, artist_id, score))

CPU times: user 3min 52s, sys: 779 ms, total: 3min 53s
Wall time: 3min 53s


In [38]:
%%time
user_plays = plays_als.T.tocsr()
with open((data_dir+"/als_default/lastfm.test.0.recs.tsv"), "w") as output_file:
    for user_id in data_Test["user_id"].unique():
        for artist_id, score in model_als_default.recommend(user_id, user_plays):
                output_file.write("%s\t%s\t%s\n" % (user_id, artist_id, score))

CPU times: user 3h 28min 59s, sys: 46 s, total: 3h 29min 45s
Wall time: 52min 28s


In [39]:
%%time
user_plays = plays_als.T.tocsr()
with open((data_dir+"/als_tune/lastfm.test.0.recs.tsv"), "w") as output_file:
    for user_id in data_Test["user_id"].unique():
        for artist_id, score in model_als_tune.recommend(user_id, user_plays):
                output_file.write("%s\t%s\t%s\n" % (user_id, artist_id, score))

CPU times: user 1h 27min 46s, sys: 30.7 s, total: 1h 28min 17s
Wall time: 22min 5s


### KNN

In [40]:
!mrec_evaluate --input_format tsv --test_input_format tsv --train $DATA_DIR/lastfm.test.0 --recsdir $DATA_DIR/knn

[2017-11-30 13:39:01,703] INFO: processing /home/spynal/DATA/lastfm-dataset-360K/lastfm.test.0...
None
mrr            0.0528 +/- 0.0000
prec@5         0.0215 +/- 0.0000
prec@10        0.0227 +/- 0.0000
prec@15        0.0151 +/- 0.0000
prec@20        0.0114 +/- 0.0000


### ALS
по умолчанию

In [41]:
!mrec_evaluate --input_format tsv --test_input_format tsv --train $DATA_DIR/lastfm.test.0 --recsdir $DATA_DIR/als_default

[2017-11-30 13:42:03,201] INFO: processing /home/spynal/DATA/lastfm-dataset-360K/lastfm.test.0...
None
mrr            0.5834 +/- 0.0000
prec@5         0.3469 +/- 0.0000
prec@10        0.3025 +/- 0.0000
prec@15        0.2017 +/- 0.0000
prec@20        0.1513 +/- 0.0000


### ALS
С подобранными параметрами

In [42]:
!mrec_evaluate --input_format tsv --test_input_format tsv --train $DATA_DIR/lastfm.test.0 --recsdir $DATA_DIR/als_tune

[2017-11-30 13:45:03,164] INFO: processing /home/spynal/DATA/lastfm-dataset-360K/lastfm.test.0...
None
mrr            0.6258 +/- 0.0000
prec@5         0.3750 +/- 0.0000
prec@10        0.3229 +/- 0.0000
prec@15        0.2153 +/- 0.0000
prec@20        0.1614 +/- 0.0000
